ISSUEs: 

what is the correct order of operations for calculating IVT st

not sure IVT convergence is being calculated correctly because the way ivt_full is structured, latitudes stay fixed while longitudes increase, and then latitude increases by one, longitudes increase with that next latitude fixed. i feel like theres an easier way to do this, by creating a meshgrid or something

In [1]:
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import cartopy.crs as ccrs
from scipy.interpolate import griddata
from matplotlib.patches import Rectangle
import cartopy.feature as cfeature
import glob
from datetime import date
import os
import re
from datetime import datetime, timedelta
from haversine import haversine, Unit
import metpy
import metpy.constants as mpconst
import metpy.units as units
from scipy.interpolate import interp1d
from scipy.optimize import bisect
import numpy as np
import pickle
import numpy as np
import math
import pint
import xarray as xr
import netCDF4 as nc
from matplotlib.colors import TwoSlopeNorm
import re
import matplotlib.pyplot as plt
from numpy import load
from numpy import asarray
from numpy import save
import pytz
from scipy import stats
from sklearn.linear_model import LinearRegression
from collections import Counter
import pymannkendall as mk
from pydoc import help
import warnings
import seaborn as sns
%matplotlib inline

# Set display options to show all rows and columns in dataframe
# pd.set_option('display.max_rows', None)

# pd.set_option('display.max_columns', None)

# np.set_printoptions(threshold=np.inf)
warnings.filterwarnings("ignore")

In [2]:
with open('soundings_fig2&3.pdkl', 'rb') as file:
    
    soundings = pickle.load(file)

In [3]:
#reading in IVT files 
#units for ivtx/ivty are kg/(m*s)

path = "../../data/rong4/Data/ERA5"

xfile = os.path.join(path, 'era5_ivtx_1979-2020_4times.nc')

yfile = os.path.join(path, 'era5_ivty_1979-2020_4times.nc')

ivtx_df = xr.open_dataset(xfile)

ivty_df = xr.open_dataset(yfile)

In [9]:
ivtx = ivtx_df['p71.162']

ivty = ivty_df['p72.162']

#selected ranges for analysis: start time between (inclusive) 2001 and 2018

#longitudes between 240 --> 290 which corresponds to 120 --> 70 W

#latitudes between 25 --> 50

start_time = np.datetime64('2001-01-01T00:00:00')

end_time = np.datetime64('2018-12-31T23:59:59')

lon_range = np.arange(220, 300)

lat_range = np.arange(25, 51)

# .sel allows you to index a variable (in this case ivt) according to specific ranges

dfx = ivtx.sel(time=slice(start_time, end_time), latitude=lat_range, longitude=lon_range).to_dataframe().reset_index()

# converting to LST

# dfx['time'] = pd.to_datetime(dfx['time']) - pd.Timedelta(hours=6)

# leaving as UTC
# dfx['utc_time'] = pd.to_datetime(dfx['time'])

dfx.rename(columns={'p71.162': 'ivtx'}, inplace=True)

dfy = ivty.sel(time=slice(start_time, end_time), latitude=lat_range, longitude=lon_range).to_dataframe().reset_index()

# dfy['time'] = pd.to_datetime(dfy['time']) - pd.Timedelta(hours=6)

# dfy['utc_time'] = pd.to_datetime(dfy['time'])

dfy.rename(columns={'p72.162': 'ivty'}, inplace=True)

#merging ivtx and ivty together

ivt_full = pd.merge(dfx, dfy, on=['time', 'latitude', 'longitude'])

#filtering only for warm season months
ivt_full = ivt_full[ivt_full['time'].dt.month.isin([5, 6, 7, 8, 9])]

ivt_full = ivt_full.reset_index(drop=True)

In [10]:
#IVT calculation 

ivt_full['ivt'] = np.sqrt(ivt_full['ivtx']**2 + ivt_full['ivty']**2)

ivt_full.rename(columns={'time': 'utc_time'}, inplace=True)

In [11]:
#there are 4 observations per day (0, 6, 12, 18)
#calculating the daily average for every grid point for ivt, ivtx and ivty
#2005-09-22 is a good example of large difference between calculating IVT average at first step (for every row) 
#or second step: meaning calculating IVT average as an average of the ivtx mean and ivty mean

ivt_full['day'] = ivt_full['utc_time'].dt.date

# ivt_full = ivt_full[ivt_full['utc_time'].dt.hour.between(0, 6)]

daily_ivt_avg = ivt_full.groupby(['day', 'latitude', 'longitude']).agg({
    
    'ivt': 'mean',
    
    'utc_time': 'size'
    
}).reset_index()

In [12]:
#climatology calculation 

daily_ivt_avg['month_day'] = daily_ivt_avg['day'].apply(lambda x: x.strftime('%m-%d'))

clim_ivt = daily_ivt_avg.groupby(['month_day', 'latitude', 'longitude']).agg({
    
    'ivt': ['mean', 'std', 'count'],
    
    'day': 'size', 
    
    

}).reset_index()

clim_ivt.set_index('month_day', inplace=True)

daily_ivt_avg.set_index('month_day', inplace=True)

clim_ivt.columns = ['_'.join(col).strip() if col[1] else col[0] for col in clim_ivt.columns.values]

In [17]:
#standardized anomaly calculation

anom_daily_ivt = pd.merge(daily_ivt_avg, clim_ivt, on=['month_day', 'latitude', 'longitude'], how='inner')

anom_daily_ivt['ivt_stnd_anom'] = (anom_daily_ivt['ivt'] - anom_daily_ivt['ivt_mean'])/anom_daily_ivt['ivt_std']

In [19]:
#daily average 

anom_daily_ivt = anom_daily_ivt[['day','latitude','longitude','ivt_stnd_anom']].reset_index()

anom_daily_ivt = anom_daily_ivt.set_index('day')

In [20]:
#changing shape so that each day is a single row 

ivt_flat = anom_daily_ivt.pivot_table(index='day', columns=['latitude', 'longitude'], values=['ivt_stnd_anom'])

ivt_flat.reset_index(inplace=True)

#flattening the MultiIndex column names

ivt_flat.columns = [''.join(map(str, col)) for col in ivt_flat.columns]

ivt_flat = ivt_flat.set_index('day')

In [23]:
with open('ivt_flat.pdkl', 'wb') as f:
    
    pickle.dump(ivt_flat, f)

## END OF SCRIPT FOR IVT STANDARDIZED ANOMALIES

In [24]:
ivt_sound = pd.merge(ivt_flat, soundings, how='inner', left_index=True, right_index=True)

In [25]:
dry_ape_ivt = ivt_sound[(ivt_sound['dry_coupling'] == True) & (ivt_sound['APE'] == True)]

wet_ape_ivt = ivt_sound[(ivt_sound['wet_coupling'] == True) & (ivt_sound['APE'] == True)]

dry_ape_ivt.shape, wet_ape_ivt.shape

((91, 2107), (78, 2107))

In [26]:
sgp_lon_min, sgp_lon_max = 262, 263

sgp_lat_min, sgp_lat_max = 36, 37

latitude_size = 26

longitude_size = 80

latitudes = np.linspace(25, 50, latitude_size)

longitudes = np.linspace(220, 299, longitude_size)

lon, lat = np.meshgrid(longitudes, latitudes)

## IVT CONVERGENCE DRAFT BELOW

In [ ]:
# IVT convergence calculation below, needs work

#ivt_full['longitude'] = ivt_full['longitude']-360

# def calculate_distances(latitudes, longitudes):
    
#     num_points = len(latitudes)
    
#     # Initialize distance arrays
#     dx = np.zeros(num_points)
    
#     dy = np.zeros(num_points)
    
#     # Calculate distances
#     for i in range(num_points):
        
#         if i < num_points-1: 
            
#             print((latitudes[i], longitudes[i]), (latitudes[i+1], longitudes[i]))
            
#             dx[i] = haversine((latitudes[i], longitudes[i]), (latitudes[i], longitudes[i+1]), unit=Unit.METERS)
            
#             dy[i] = haversine((latitudes[i], longitudes[i]), (latitudes[i+1], longitudes[i]), unit=Unit.METERS)
            
#         if i == num_points-1: 
            
#             dx[i] = haversine((latitudes[i], longitudes[i]), (latitudes[i], longitudes[i-1]), unit=Unit.METERS)
            
#             dy[i] = haversine((latitudes[i], longitudes[i]), (latitudes[i-1], longitudes[i]), unit=Unit.METERS)
    
#     return dx, dy


# def calculate_convergence(df):

#     dx, dy = calculate_distances(df['latitude'], df['longitude'])
    
#     df['dx'] = dx
    
#     df['dy'] = dy
    
#     # central differences for inner points
#     for i in range(1, len(df) - 1):
        
#         if df.loc[i, 'dx'] != 0: 
        
#             d_ivtx_dx = (df.loc[i + 1, 'ivtx'] - df.loc[i - 1, 'ivtx']) / (2 * df.loc[i, 'dx'])
            
#         else: 
            
#             d_ivtx_dx = 0
            
#         if df.loc[i, 'dy'] != 0: 
            
#             d_ivty_dy = (df.loc[i + 1, 'ivty'] - df.loc[i - 1, 'ivty']) / (2 * df.loc[i, 'dy'])
            
#         else:
            
#             d_ivty_dy = 0
        
#         df.loc[i, 'convergence'] = -(d_ivtx_dx + d_ivty_dy)
    
#     # Forward difference for the first row
    
#     if df.loc[1, 'dx'] != 0: 
        
#         d_ivtx_dx = (df.loc[1, 'ivtx'] - df.loc[0, 'ivtx']) / df.loc[0, 'dx']
        
#     else: 
        
#         d_ivtx_dx = 0
    
#     if df.loc[1, 'dy'] != 0: 
        
#         d_ivty_dy = (df.loc[1, 'ivty'] - df.loc[0, 'ivty']) / df.loc[0, 'dy']
        
#     else: 
        
#         d_ivty_dy = 0 
    
#     df.loc[0, 'convergence'] = -(d_ivtx_dx + d_ivty_dy)
    
#     # Backward difference for the last row
    
#     if df.loc[len(df)-1, 'dx'] != 0:
    
#         d_ivtx_dx = (df.loc[len(df) - 1, 'ivtx'] - df.loc[len(df) - 2, 'ivtx']) / df.loc[len(df) - 1, 'dx']
        
#     else: 
        
#         d_ivtx_dx = 0 
    
#     if df.loc[len(df)-1, 'dy'] != 0:
        
#         d_ivty_dy = (df.loc[len(df) - 1, 'ivty'] - df.loc[len(df) - 2, 'ivty']) / df.loc[len(df) - 1, 'dy']
        
#     else: 
        
#         d_ivty_dy = 0 
    
#     df.loc[len(df) - 1, 'convergence'] = -(d_ivtx_dx + d_ivty_dy)
    
#     return df


# df = pd.DataFrame({
    
#     'latitude': [25, 25, 25, 25, 26, 27, 28, 39],
    
#     'longitude': [-70, -80, -90, -100, -120, -120, -120, -120],
    
#     'ivtx': [10000, 20000, 30000, 45000, 30000, 60000, 10000, 12000],
    
#     'ivty': [50000, 10000, 15000, 20000, 12000, 13000, 14000, 90000]
# })

# df = calculate_convergence(df)

# print(df)


## IVT vector example for a specific date time

In [ ]:
# # Filter data for the specific timestamp
# #why are vectors so long near the high amount of moisture?

# time_stamp = pd.to_datetime('2014-08-26 18:00:00')

# ivt_sample = ivt_full[ivt_full['utc_time'] == time_stamp]

# # Define grid for pcolormesh
# lon_grid, lat_grid = np.meshgrid(np.linspace(ivt_sample['longitude'].min(), ivt_sample['longitude'].max(), 100),
                                 
#                                  np.linspace(ivt_sample['latitude'].min(), ivt_sample['latitude'].max(), 100))

# # Interpolate IVT data onto grid
# ivt_grid = griddata((ivt_sample['longitude'], ivt_sample['latitude']), ivt_sample['ivt'], (lon_grid, lat_grid), method='linear')

# # Get IVT components
# ivtx = ivt_sample['ivtx'].values

# ivty = ivt_sample['ivty'].values

# # Get unique longitudes and latitudes for quiver plot
# lon = ivt_sample['longitude'].unique()

# lat = ivt_sample['latitude'].unique()

# # Plotting
# plt.figure(figsize=(20, 15))

# ax = plt.axes(projection=ccrs.PlateCarree())

# # Plot smoothed IVT data using pcolormesh
# pcm = ax.pcolormesh(lon_grid, lat_grid, ivt_grid, vmin=0, vmax=np.max(ivt_sample['ivt']), cmap='Blues', shading='auto', transform=ccrs.PlateCarree())

# # Add quiver plot for vectors
# q = ax.quiver(ivt_sample['longitude'], ivt_sample['latitude'], ivtx, ivty, color='black', scale_units='xy', scale=70, width=0.001, headwidth=3, headlength=1, headaxislength=4.5, transform=ccrs.PlateCarree())

# # Add quiver key
# ax.quiverkey(q, X=0.9, Y=1.05, U=150, label='example vector = 150 kg*m-1*s-1', labelpos='E')

# # Colorbar
# plt.colorbar(pcm, label='IVT', shrink=0.3)

# # Set labels and add coastlines for context
# ax.set_xlabel('Longitude')
# ax.set_ylabel('Latitude')

# # Add state borders
# states_provinces = cfeature.NaturalEarthFeature(category='cultural',
#                                                 name='admin_1_states_provinces_lines',
#                                                 scale='50m',
#                                                 facecolor='none')

# ax.add_feature(states_provinces, edgecolor='gray')
# ax.add_feature(cfeature.BORDERS)
# ax.add_feature(cfeature.STATES, edgecolor='gray')
# ax.coastlines()

# # Show plot
# plt.show()
